<a href="https://colab.research.google.com/github/nandini933/Java/blob/main/vehicle_traffic_congestion_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import cv2
import numpy as np
import urllib.request
import os

# Function to download YOLO files
def download_yolo_files():
    yolov3_files = [
        ("yolov3.weights", "https://pjreddie.com/media/files/yolov3.weights"),
        ("yolov3.cfg", "https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true"),
        ("coco.names", "https://github.com/pjreddie/darknet/blob/master/data/coco.names?raw=true")
    ]

    for filename, url in yolov3_files:
        if not os.path.isfile(filename):
            print(f"Downloading {filename}...")
            urllib.request.urlretrieve(url, filename)
        else:
            print(f"{filename} already exists.")

# Download YOLO files if not present
download_yolo_files()
from google.colab.patches import cv2_imshow
# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getUnconnectedOutLayersNames()

# Open video capture
cap = cv2.VideoCapture("video.mp4", cv2.CAP_ANY)


# Set video frame size
cap.set(3, 1280)
cap.set(4, 720)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, channels = frame.shape

    # Convert the frame to a blob
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Set the input to the network
    net.setInput(blob)

    # Run forward pass and get output
    outs = net.forward(layer_names)

    # Lists to store detected vehicles
    class_ids = []
    confidences = []
    boxes = []

    # Iterate through each detection
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5 and class_id == 2:  # Class ID for cars is 2
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Apply non-maximum suppression to eliminate redundant overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and count vehicles
    vehicle_count = 0
    for i in indices:
        box = boxes[i]
        x, y, w, h = box

        # Draw rectangle
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        vehicle_count += 1

    # Display vehicle count
    cv2.putText(frame, f'Vehicle Count: {vehicle_count}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the resulting frame
    cv2_imshow(frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
pip install --upgrade opencv-python
